# generate_fake_dataset

Filterを学習するためのfake datasetをGeneratorで生成する。

In [1]:
import os
import pickle
import numpy as np
import tensorflow as tf

In [2]:
# モデルパラメータ
num_layers = 4
d_model = 128
dff = 512
num_heads = 8

GENERATOR_EPOCH = 37

# 生成パラメータ
TEMPERATURE = 1.0
BATCH_SIZE = 256

In [3]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
sp.load('sentencepiece.model')

True

In [4]:
# weightファイル読み込みのために利用する
with open("real_dataset.pickle", "rb") as f:
    ids = pickle.load(f)
real_dataset_tensor = tf.keras.preprocessing.sequence.pad_sequences(ids, padding='post')

In [5]:
# 入力データのパラメータ
vocab_size = sp.get_piece_size()
seq_len = real_dataset_tensor.shape[1]

In [6]:
from scripts.model import Generator

In [7]:
generator = Generator(num_layers, d_model, num_heads, dff, vocab_size, max_pos_encoding=seq_len)

In [8]:
# HACK: create variables
_ = generator(tf.constant(real_dataset_tensor[:1]), training=False)

In [9]:
generator.load_weights(f'model/generator/weights_epoch{GENERATOR_EPOCH}.h5')

In [ ]:
# generate with initial model
generation_ids = generator.sample(num_sample=50, temperature=TEMPERATURE, padding=True)

for ids in generation_ids:
    ids_int = list(map(lambda x: int(x), ids))
    print(sp.decode_ids(ids_int))

In [11]:
# real_datasetと同じ数のfake_datasetを作成する
NUM_BATCH = len(real_dataset_tensor) // BATCH_SIZE + 1
NUM_BATCH

6671

In [12]:
fake_dataset = []
for i in range(NUM_BATCH):
    generation_ids = generator.sample(num_sample=BATCH_SIZE, temperature=TEMPERATURE, padding=True)
    if i % 100 == 0:
        print(i)
    fake_dataset.append(generation_ids)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600


In [13]:
fake_dataset = np.vstack(fake_dataset)

In [14]:
# save generations
with open("fake_dataset.pickle", mode="wb") as f:
    pickle.dump(fake_dataset, f, protocol=pickle.HIGHEST_PROTOCOL)